# IIC2115 - Programación como herramienta para la ingeniería

## Análisis de redes
En este capítulo abordaremos el análisis de **redes** como una herramienta para representar y resolver problemas que implican relaciones o interacciones entre distintas entidades, tales como redes de transporte, redes sociales, sistemas eléctricos o de comunicación. Para ello, utilizaremos la abstracción de los **grafos**, compuestos por **nodos** (entidades) y **aristas** (conexiones) que nos permiten modelar la estructura y comportamiento de estos sistemas de forma simple y analizable.

En el ámbito práctico, aplicaremos estas ideas utilizando las librerías **NetworkX** y **OSMnx**, que permiten construir, manipular y analizar grafos de manera eficiente a partir de datos reales, como los obtenidos desde **OpenStreetMap**. Con estas herramientas utilizaremos algoritmos clásicos, por ejemplo, de rutas óptimas, conectividad o flujo máximo, y visualizaremos los resultados tanto sobre mapas geográficos como en representaciones topológicas.

### Creando nuestras propias redes ([referencia](https://networkx.org/documentation/stable/tutorial.html))

Una de las funciones de networkX es que nos permite crear nuestras propias redes. A continuación veremos el proceso de creación de una red en networkX.

In [ ]:
# Primero importamos la librería de networkX
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# Creamos una red (objeto de nx) vacía
G = nx.Graph()

Por definición un **`Graph`** es una colección de nodos (vertices) relacionados con pares de nodos (arcos). Ahora procederemos a agregar nodos y arcos a nuestra red.
#### a) Nodos

El `graph` puede ser poblado de varias formas. La librería incluye generadores de redes o funciones para leer redes desde varios formatos. Por ahora, solo incluíremos un nodo de forma sencilla.

In [ ]:
# Agregamos un solo nodo
G.add_node(1)

# Agregamos una lista de nodos
G.add_nodes_from([2, 3])

# Agregamos una lista de tuplas de nodos con atributos -> (nodo, dict de atributos)
G.add_nodes_from([(4, {'nombre': 'Santiago'}),
                  (5, {'nombre': 'Valparaíso'})])

#### b) Arcos
Para agregar los arcos, es necesario ingresar los nodos previamente. Para ingresar los arcos, debemos indicar que nodos buscamos conectar. Por ejemplo:

In [ ]:
# Agregamos un arco entre el nodo 1 y el nodo 2
G.add_edge(1, 2)

# Agregamos un arco entre el nodo 2 y 3 almacenado en una tupla
e = (2, 3)
G.add_edge(*e) # <-- Asi desempaquetamos una tupla como inputs

# Agregamos un arco 1->2 y 1->3 desde una lista
G.add_edges_from([(2, 4), (3, 5), (3,1)])

### Visualizando redes

Para poder visualizar las redes de NetworkX, podemos hacer uso del método `draw()`. Este genera una visualización de la red y si no se han indicado coordenadas optimiza la localización de nodos.

In [ ]:
nx.draw(G, with_labels=True, font_weight='bold')

### Tipos de grafos
Los grafos son conjuntos de nodos y arcos que poseen cierta estructura, en NetworkX existen 4 tipos de grafos basados en 2 categorías:
1. Grafo dirigido o no dirigido: Los arcos en NetworkX pueden tener dirección (grafo dirigido) o no tenerla (grafo no dirigido), en el caso del grafo dirigido, el arco AB es distinto al arco BA, mientras que en el caso no dirigido son exactamente lo mismo, por lo que si existe el arco AB también existe el arco BA.
2. Normal o multi grafo: Los multigrafos en NetworkX permiten tener más de un arco entre un par de nodos, en cambio los grafos normales simplemente tienen un arco para cada par de nodos.

De estas definiciones se obtienen los 4 grafos existentes en NetworkX:
- Graph
- DiGraph
- MultiGraph
- MultiDiGraph

### Creación de grafo a partir de fuentes de datos

Ahora que ya sabemos lo básico de la estructura de una red o grafo, es posible crear un grafo en NetworkX mediante una fuente de datos externa. En este ejemplo, utilizaremos un *string*, el que por ejemplo, puede venir almacenado dentro de un archivo de texto. Este *string* tiene forma de tablero formado por casillas **"X"** y **"O"**. Cada casilla **"O"** corresponde a nodo y tiene por vecinos a los nodos que sean vecinos en el tablero. Por ejemplo:

```python
XOXOO
OOXOX
XOOOO
```

Primero definimos nuestra propia estructura de datos, como lo trabajamos en el capítulo 2:

In [ ]:
class Nodo:
    _id = 0

    # Constructor de un Nodo mediante un ID, su posicion dentro del tablero y una lista de los nodos vecinos
    def __init__(self, posicion):
        self._id = Nodo._id
        self.posicion = posicion # Tupla (x,y)
        self.vecinos = []
        Nodo._id += 1

    # Metodo que agrega un vecino a la lista de vecinos
    def add_vecino(self, nodo):
        self.vecinos.append(nodo)

    def __str__(self):
        return "Nodo: " + str(self.posicion) + " Vecinos: " + str([nodo.posicion for nodo in self.vecinos])

    def __repr__(self):
        return str(self._id)


Luego creamos un método que es capaz de convertir nuestra red codificada como tablero en una red usando nuestra estructura de datos

In [ ]:
def crear_grafo(matriz):
    grafo = dict()
    fila = len(matriz)
    columna = len(matriz[0])

    for i in range(fila):
        for j in range(columna):
            if matriz[i][j] == "O":
                if (i,j) not in grafo.keys():
                    nodo = Nodo((i,j))
                    grafo.update({(i,j):nodo}) # grafo[(i,j)] = nodo

                for f,c in [(-1,0),(0,-1),(1, 0), (0, 1)]: #Accede arriba, izquierda, abajo, derecha
                    if i + f >= 0 and j + c >= 0 and i + f < fila and j + c < columna: # si no estoy en un borde
                        if matriz[i+f][j+c] == "O":
                            if (i+f,j+c) not in grafo.keys():
                                vecino = Nodo((i+f,j+c))
                                grafo.update({(i+f,j+c):vecino}) # grafo[(i+f,j+c)] = vecino
                            else:
                                vecino = grafo[(i+f,j+c)]
                            grafo[(i,j)].add_vecino(vecino)
    return grafo

In [ ]:
tablero = """XOXOO
OOXOX
XOOOO
"""
filas_tablero = tablero.split()
grafo = crear_grafo(filas_tablero)

for nodo in grafo.values():
    print(nodo)

Es posible crear una visualización de nuestra red. Pero qué pasa si ahora utilizamos NetworkX. Crearemos un método que define una red de NetworkX, y luego la graficaremos.

In [ ]:
def crear_grafo_nx(grafo):

    # Esta función crea un grafo no dirigido de networkx utilizando la estructura de datos creada anteriormente
    G = nx.Graph()
    #posiciones = dict()
    arcos = set()

    for nodo in grafo.values():
        G.add_node(nodo._id,pos = nodo.posicion)
        #posiciones[nodo._id] = nodo.posicion
        for vecino in nodo.vecinos:
            if (nodo._id, vecino._id) not in arcos and (vecino._id, nodo._id) not in arcos: #se crea solo si no existe
                G.add_edge(nodo._id, vecino._id) #cola, cabeza
    return G

In [ ]:
def graficar_red(G, tamaño_fuente, tamaño_nodo):
    posiciones = nx.get_node_attributes(G,'pos')
    nx.draw(G, pos=posiciones, with_labels=True, font_size=tamaño_fuente, node_size=tamaño_nodo, node_color="lightblue")
    plt.show()

In [ ]:
graficar_red(crear_grafo_nx(grafo),20,1000)

Ahora realicemos lo mismo con una red mucho más grande

In [ ]:
def leer_datos(path):
    with open(path,"r", encoding = "UTF-8") as archivo:
        matriz = [ i.strip("\n") for i in archivo.readlines()] # i.strip("\n") para quitar los enter en cada linea
        return matriz

In [ ]:
matriz = leer_datos("mapa.txt")
Nodo._id = 0
grafo2 = crear_grafo(matriz)
for nodo in grafo2.values():
    print(nodo)

In [ ]:
graficar_red(crear_grafo_nx(grafo2),20,1000)

### Algoritmos en redes

Netowork presenta una seríe de algoritmos y problemas de optimización que pueden ser resuletos mediante el uso de librería. Dentro de la documentación pueden revisar muchos de estos algoritmos. Como rutas mínimas, flujo máximo, entre otros.

### Obteniendo redes reales

Muchas veces lo que necesitamos es obtener la redes de transporte de ciudades, o lugares específicos sin la necesidad de crearlas nosotros mismos. Para eso existe una librería que esta construida con networkX y *Open Street Maps* (OSM) que nos ayuda en la creación y procesamiento de redes de transporte reales. En este caso nos referimos a la librería osmnx. La instalan mediante pip como cualquier librería convencional: `!pip install osmnx`.

In [ ]:
!pip install osmnx

In [ ]:
# importamos la librería
import osmnx as ox

In [ ]:
# Por ejemplo, podemos descargar las redes de alguna ciudad
G = ox.graph_from_place('Macul')

In [ ]:
# Y luego graficarla
ox.plot_graph(G, figsize = (15,15), bgcolor = 'w', node_color = 'red', edge_color = 'black', node_alpha = 0);

Las redes de lugares pueden ser obtenidas de varias formas:

- Definiendo los bordes de un rectángulo. (Bounding box)
- Indicando un radio a partir de una coordenada en Lat-Long.
- Entregando un polygono
- Mediante nombres de lugares (o lista de nombres de lugares)

Además, se pueden especificar diferentes tipos de redes con el argumento `network_type`, por ejemplo:

- `drive`: Obtiene todas las calles públicas.
- `drive_service`: Obtiene todas las calles públicas, incluyendo vías de servicio.
- `walk`: Obtiene todos los tramos, vias o calles donde pueden transitar peatones.
- `bike`: Obtiene todas las vías donde pueden transitar ciclistas.
- `all`: Descarga todos los tramos de acceso público.
- `all_private`: Descarga todos los tramos de acceso público, incluidas las vías privadas.

#### Descarga desde un bounding box

Ahora descargaremos una red según un rectángulo y de tipo 'drive'. El formato para la definición del rectángulo tiene el siguiente formato: (longitud_izquierda, latitud_inferior, longitud_derecha, latitud_superior).

In [ ]:
G = ox.graph_from_bbox(bbox=(-74.03, 40.68, -73.92, 40.88), network_type='drive') # estas son coordenadas de alguna parte del mundo, pueden ponerlas en google maps

In [ ]:
G_projected = ox.project_graph(G)
ox.plot_graph(G_projected, figsize = (15,15), bgcolor = 'w', node_color = 'red', edge_color = 'black', node_alpha = 0.5, node_size = 5);

Adicionalemente, utilizando el argumento `custom_filter`, es posible especificar de manera aún más precisa los elementos que se incluirán en el mapa. Por ejemplo, si queremos descargar únicamente las calles primarias y secundarias de Macul, donde además es posible caminar , podemos utilizar el siguiente comando:

In [ ]:
G = ox.graph.graph_from_place('Macul', custom_filter='["highway"~"primary|secondary"]', network_type='walk')

Manejar este argumento puede ser muy útil para reducir la complejidad de los ejericios y el tiempo de descarga de los mapas. Más detalles del uso de `custom_filter` puede encontrarse [acá](https://github.com/gboeing/osmnx-examples/blob/main/notebooks/08-custom-filters-infrastructure.ipynb).

In [ ]:
ox.plot_graph(G, figsize = (10,10), bgcolor = 'w', node_color = 'red', edge_color = 'black');

### Ruteo en redes

Dado que osmnx esta escrito con las ventajas de networkx, es posible ejecutar todos los método de netowrkx con las redes que obtengamos de osmnx. Uno de ellos, es la posibilidad de hacer ruteo. Veamos un ejemplo:

In [ ]:
# Buscamos dos puntos que generen un recatangulo (coordenadas obtenidos desde Rancagua en Gmaps)
# OSMNX también les permite obtener las coordenadas de direcciónes
izq = -70.78
der = -70.69
arriba = -34.12
abajo = -34.20

In [ ]:
# Obtenemos la red

net = ox.graph_from_bbox(bbox=(izq, abajo, der, arriba), network_type= 'drive')
ox.plot_graph(net, figsize = (20,20), node_size = 10, bgcolor = 'w', node_color = 'red', edge_color = 'black')

In [ ]:
# Buscamos un Origen y un destino (ejemplos)

orig_node = ox.distance.nearest_nodes(net, -70.746707, -34.164603)
dest_node = ox.distance.nearest_nodes(net, -70.711568, -34.180152)

In [ ]:
# Realizamos ruteo

route = nx.shortest_path(net, orig_node, dest_node, weight = 'length')
cost = nx.shortest_path_length(net, orig_node, dest_node, weight = 'length')
fig, ax = ox.plot_graph_route(net, route, node_size = 10, figsize=(20,20), bgcolor = 'w', node_color = 'blue', edge_color = 'black')

### Convertir redes a datos geoespaciales (para GeoPandas)

Podemos guardar nuestras redes como archivos manejables por GeoPandas y luego abrirlos con esta librería.

In [ ]:
ox.save_graph_geopackage(net, "graph.gpkg")

In [ ]:
import geopandas as gpd

df_nodos = gpd.read_file('graph.gpkg', layer="nodes")
df_arcos = gpd.read_file('graph.gpkg', layer="edges")

In [ ]:
df_nodos.plot()

In [ ]:
df_arcos.plot()

Ademas podemos consultar la información asociada a los nodos y arcos:

In [ ]:
df_nodos.head()

In [ ]:
df_arcos.head()